In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisosONEHOT.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantesONEHOT.csv")
areas=avisos['nombre_area'].value_counts()
avisos['nombre_area']=avisos['nombre_area'].replace(areas.index, areas.values)

avisos_no_one_hot= pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
avisos_no_one_hot=avisos_no_one_hot[['idaviso','tipo_de_trabajo','nivel_laboral','nombre_area']];
avisos_no_one_hot['tipo_de_trabajo']=avisos_no_one_hot['tipo_de_trabajo'].astype('category')
avisos_no_one_hot['nivel_laboral']=avisos_no_one_hot['nivel_laboral'].astype('category')
avisos_no_one_hot['nombre_area']=avisos_no_one_hot['nombre_area'].astype('category')
avisos_no_one_hot=avisos_no_one_hot.rename(columns={'nombre_area': 'nombre_area_aux'})

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16501975


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,avisos_no_one_hot,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,cantidad
15931239,1112498161,2zQLLbY,0,26.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Part-time,Senior / Semi-Senior,Ventas,NaN
4904490,1112363768,JBXOj53,1,30.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Comercial,0.050000
12753547,1112249433,akONa05,0,31.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Comercial,NaN
2481994,1112052295,8QYJ0W,0,32.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Consultoria,NaN
14848686,1111305089,DrzAwJW,0,23.0,True,False,False,False,False,False,...,False,False,False,True,False,False,Full-time,Junior,Administración de Personal,NaN
12105799,1112384412,JBePrlb,0,30.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Planeamiento económico-financiero,NaN
6930886,1112380204,xkd401K,1,38.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Comercial,0.500000
5368353,1112455533,ZDrqZBY,0,19.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Compras,NaN
1153713,1112432317,83q9mz,1,41.0,True,False,False,False,False,False,...,False,False,False,False,False,False,Full-time,-,Marketing,NaN
3016435,1112386615,vVeXdxj,1,23.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Créditos y Cobranzas,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,cantidad
14994504,1112463694,mzVL3m3,1,32.0,False,True,False,False,False,False,...,False,False,False,True,False,False,Full-time,Junior,Ventas,0.250000
1833433,1112262617,6RlYWv,0,43.0,True,False,False,False,False,False,...,False,False,True,False,False,False,Full-time,Jefe / Supervisor / Responsable,Ingeniería Oficina Técnica / Proyecto,0.000000
14955540,1112406946,axGpM0,0,40.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Maternidad,0.000000
8618128,1112358166,JBxKWVJ,0,27.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Call Center,0.000000
5501148,1112425141,QNawOxo,1,31.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Recursos Humanos,0.013514
7398676,1112471982,OqrZ3ON,0,22.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Administración de Personal,0.000000
6996955,1112258186,OqrxO36,1,33.0,True,False,False,False,False,False,...,False,False,False,True,False,False,Part-time,Junior,Telemarketing,0.000000
1612407,1112380729,NzrZ6RL,1,25.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Almacén / Depósito / Expedición,0.000000
2914528,1112145679,akxQdPE,1,22.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Call Center,0.307692
790236,1112399604,63Xajl,0,42.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Comunicaciones Externas,0.000000


In [15]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area_aux'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area_aux,postulaciones_misma_area
0,0z5Dmrd,Abastecimiento,1
1,0z5Dmrd,Administración,2
2,0z5Dmrd,Administración de Personal,1
3,0z5Dmrd,Almacén / Depósito / Expedición,1
4,0z5Dmrd,Atención al Cliente,1


In [16]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,2
1,0z5Dmrd,Junior,4
2,0z5Dmrd,Otro,1
3,0z5Dmrd,Senior / Semi-Senior,18
4,0z5JW1r,Gerencia / Alta Gerencia / Dirección,2


In [17]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,24
1,0z5Dmrd,Part-time,1
2,0z5JW1r,Full-time,26
3,0z5VvGv,Full-time,58
4,0z5VvGv,Part-time,11


In [18]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
4114548,1112401940,YjlbYXJ,1,33.0,True,False,False,False,False,False,...,False,True,False,Full-time,Otro,Comercial,0.000000,0,41,5
3227252,1112410555,6rP12vx,1,200.0,False,False,True,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Tesorería,0.000000,16,281,191
14170901,1112417473,8MZmY0W,0,26.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Programación,0.000000,3,37,25
4102491,1112336465,kP6BzxQ,0,25.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Recursos Humanos,0.000000,0,26,22
1799045,1112033906,wVpr8NV,1,27.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Ventas,0.009259,1,75,52
13075705,1112308574,QNP6pAz,1,31.0,False,True,False,False,False,False,...,True,False,False,Full-time,Junior,Almacén / Depósito / Expedición,0.000000,19,348,95
6704176,1112361348,xkPdbGa,1,29.0,True,False,False,False,False,False,...,True,False,False,Full-time,Junior,Impuestos,0.008850,9,149,40
98851,1112348995,wVkVEoQ,1,21.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Recepcionista,0.000000,0,30,26
5997850,1112445120,eAxXp4,0,39.0,True,False,False,False,False,False,...,False,False,False,Full-time,-,Liderazgo de Proyecto,0.000000,0,20,1
9491452,1112429410,bOjBWw0,0,29.0,True,False,False,False,False,False,...,False,False,False,Full-time,Jefe / Supervisor / Responsable,Administración,0.000000,2,18,0


In [19]:
train, test=train_test_split(postulaciones.sample(2*(len(postulaciones)//3)),test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')
features.remove('nombre_area_aux')
features.remove('tipo_de_trabajo')
features.remove('nivel_laboral')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  10187895 Test:  536205


In [20]:
c=0
gc.collect()

88

In [21]:
len(features)

75

In [22]:
c=MLPClassifier(activation='tanh', alpha=1e-05, batch_size='auto',beta_1=0.9, beta_2=0.999,epsilon=1e-08, hidden_layer_sizes=(len(features), 6), learning_rate='constant',learning_rate_init=0.001, max_iter=200,nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,solver='adam', tol=0.0001, validation_fraction=0.1, verbose=True,warm_start=False)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/NN.pkl")
dt=c.fit(x_train,y_train)

Iteration 1, loss = 0.30244990
Iteration 2, loss = 0.28180093
Iteration 3, loss = 0.27727476
Iteration 4, loss = 0.28321312
Iteration 5, loss = 0.28209774
Iteration 6, loss = 0.28092439
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


In [23]:
score=c.score(x_test,y_test)*100
print(score)

88.60659635773631


In [24]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/NN.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/NN.pkl']

In [25]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos_no_one_hot,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.sample(10)

100000
100000
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
79612,27086,1112347658,ZZZ8KV,45.0,False,True,False,False,False,False,...,False,False,False,Full-time,Jefe / Supervisor / Responsable,Producción,0.000000,2.0,34.0,2.0
56528,44528,1112426721,NpRpxz,54.0,True,False,False,False,False,False,...,True,False,False,Full-time,Junior,Planeamiento comercial,0.000000,0.0,19.0,1.0
40068,57158,1112451106,ZxOJob,56.0,False,True,False,False,False,False,...,True,False,False,Part-time,Junior,Ventas,0.000000,5.0,4.0,6.0
22864,18615,1112213212,X9xmb0P,23.0,False,True,False,False,False,False,...,False,False,False,Full-time,-,Seguros,0.023810,1.0,20.0,1.0
33541,8045,1111753681,zvwremJ,26.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Tesorería,0.000000,3.0,31.0,23.0
12849,46547,1112430193,ZvJLQP,33.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Camareros,0.019608,1.0,25.0,21.0
92741,91822,1112473201,8wKRKL,47.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Ingeniería Civil,0.000000,0.0,17.0,6.0
99108,99033,1112509164,ZD3BBP,56.0,False,True,False,False,False,False,...,True,False,False,Part-time,Junior,Otros,0.000000,0.0,0.0,3.0
78811,26162,1112338157,5P0AAk,35.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Mantenimiento,0.000000,0.0,22.0,19.0
47424,74097,1112463598,6PA1Ev,33.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Programación,0.000000,0.0,19.0,16.0


In [26]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.97978215, 0.02021785],
       [0.39455456, 0.60544544],
       [0.9699089 , 0.0300911 ],
       ...,
       [0.91948245, 0.08051755],
       [0.83627098, 0.16372902],
       [0.97822487, 0.02177513]])

In [27]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [28]:
prediccion['sepostulo']=y_final2

In [29]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,False,False,Full-time,Jefe / Supervisor / Responsable,Comercial,0.0,1.0,21.0,0.0,0.020218
1,1,739260,6v1xdL,30.0,False,True,False,False,False,False,...,False,False,Full-time,Jefe / Supervisor / Responsable,Comercial,0.0,6.0,59.0,0.0,0.605445
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,False,False,Full-time,Jefe / Supervisor / Responsable,Comercial,0.0,3.0,19.0,1.0,0.030091
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,True,False,Full-time,Otro,Tecnologia / Sistemas,0.0,2.0,20.0,0.0,0.029185
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,True,False,Full-time,Otro,Tecnologia / Sistemas,0.0,1.0,16.0,2.0,0.011269


In [30]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.drop(['tipo_de_trabajo','nivel_laboral','nombre_area_aux'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.020218
1,1,0.605445
2,2,0.030091
3,3,0.029185
4,4,0.011269


In [31]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion40.csv",index=False)

In [32]:
prediccion.sepostulo.mean()

0.4596343266725699